### Setup

In [ ]:
# install if not yet done
using Pkg

# Pkg.add(url="https://github.com/mppmu/BoostFractor.jl.git")
# Pkg.add(url="https://github.com/bergermann/Dragoon.jl.git")
Pkg.update()

In [ ]:
using Dragoon, Plots #, Dates

In [ ]:
n = 20 # number of discs in the booster

initdist = findpeak1d(22.025e9,n) # initial disc configuration

freqs = genFreqs(22.025e9,50e6; n=10) # optimize on these frequencies
freqsplot = genFreqs(22.025e9,150e6; n=1000) # plot with these frequencies

booster = AnalyticalBooster(initdist; ndisk=n)  # physical properties of the booster

hist = initHist(booster,10000,freqs,ObjAnalytical); # stores measurements
#                           ^ length, should be at least n

### Optimizer

In [ ]:
trace = simulatedAnnealing(booster,hist,freqs,
        100e-6,
        TempLinear(100,10001),
        ObjAnalytical,
        UnstuckDont;
        maxiter=Int(10001),
        nreset=500,
        nresetterm=10,
        showtrace=true,
        showevery=1000,
        unstuckisiter=true,
        traceevery=100);

In [ ]:
#obtain result, booster is now in optimized position
#pos2dist(booster.pos)     #in distance space
booster.pos               #in position space

In [ ]:
#get handy output
analyse(hist,trace[1:end-1],freqsplot; freqs=freqs,div=20,scale=1e9,ylim=[-0.05e4,3e4])

#savefig(plt[i],"cool_epic_result_wow.svg")

In [ ]:
booster.timestamp